In [5]:
import gentrl
import torch
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import random
from utilities.config import Config
from rdkit import Chem

torch.cuda.set_device(0)

In [6]:
from moses.metrics import mol_passes_filters, QED, SA, logP
from moses.metrics.utils import get_n_rings, get_mol

def get_num_rings_6(mol):
    r = mol.GetRingInfo()
    return len([x for x in r.AtomRings() if len(x) > 6])


def penalized_logP(mol_or_smiles, masked=False, default=-5):
    mol = get_mol(mol_or_smiles)
    if mol is None:
        return default
    reward = logP(mol) - SA(mol) - get_num_rings_6(mol)
    if masked and not mol_passes_filters(mol):
        return default
    return reward

In [7]:
try:
    mol = Chem.MolFromSmiles('CCOc1ncc(C(O)C(=O)Nc2nnc(C)cc2)s1')
except:
    print("CC1CC=C(C(=O)N1Cc1ccc2c(c1)OCCO2")

In [8]:
config = Config(smiles_format=2, topn_fp_features=5, mode='threshold', max_fp_features=2048, threshold=0.3, morgan_radius=2)
with open(config.FpsSOM_model, 'rb') as infile:
    fps_som = pickle.load(infile)

In [10]:
enc = gentrl.RNNEncoder(latent_size=50)
dec = gentrl.DilConvDecoder(latent_input_size=50)
model = gentrl.GENTRL(enc, dec, 50 * [('c', 20)], [('c', 20)], beta=0.001)
model.cuda()

model.load('saved_gentrl_LA/')
model.cuda();

In [11]:
model.train_as_rl(fps_som.som_reward)

!Error: 0


RDKit ERROR: [22:04:29] SMILES Parse Error: unclosed ring for input: 'OC(CNc1ccccc1CO)c1[nH]nc2c1c[nH]c(=N2CCOC3)c1'


KeyError: 0

In [ ]:
model.save('./saved_gentrl_LA_after_rl/')